<a href="https://colab.research.google.com/github/puteriirenealia/Culinary-Compass/blob/main/Culinary_Compass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🍽️ Culinary Compass (AI-Powered Restaurant Review Analysis System)**
**A Comprehensive GenAI Study Jam 2025 Capstone Project**

**Please avoid using 'Run All' and execute each code segment individually, as you will be prompted to enter your OpenAI API Key 🔑 and Google Maps API Key 🔑.**

**🎯 Project Overview**
Welcome to my Kaggle Gen AI Study Jam Capstone Project! This project demonstrates the practical application of three core Generative AI capabilities through the lens of restaurant review analysis. By combining real-world data from Google Maps with cutting-edge AI techniques, we'll build an intelligent system that can understand, analyze, and provide insights about dining experiences.

**🚀 What This Project Accomplishes**
In today's digital age, restaurant reviews are everywhere, but making sense of hundreds of opinions can be overwhelming. This project tackles that challenge by leveraging the power of Generative AI to:


* Understand sentiment behind customer reviews using few-shot learning
* Summarize key insights from multiple reviews with structured AI output
* Answer specific questions about restaurants using Retrieval-Augmented Generation (RAG)

# 1: Installing and Importing Necessary Libraries
This section sets up our development environment by installing required packages and importing the essential libraries for our AI-powered restaurant review analysis system.

Key packages being installed:

**langchain** - Core framework for building AI applications with language models\
**openai** - Official OpenAI API client for GPT models\
**faiss-cpu** - Facebook's efficient similarity search library for vector embeddings\
**pandas** - Data manipulation and analysis library\
**plotly**- Interactive visualization library for creating charts and graphs\
**langchain-community** - Community-contributed LangChain integrations\
**langchain_openai** - Official OpenAI integrations for LangChain

In [2]:
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install pandas
!pip install plotly
!pip install langchain-community
!pip install langchain_openai

In [3]:
#Library Imports
#Standard Python Libraries

import os          #Operating system interface for environment variables
import json        #JSON data parsing and manipulation
import requests    #HTTP library for making API calls to Google Maps
from typing import List, Dict    #Type hints for better code documentation
from dataclasses import dataclass    #Decorator for creating data classes
from getpass import getpass    #Secure password input without echoing to screen

In [4]:
#Langchain Core Components
#Language Model Integration
from langchain_openai import ChatOpenAI    #OpenAI's GPT models interface

#Prompt Engineering Tools
from langchain.prompts import PromptTemplate, FewShotPromptTemplate    #Template systems for structured prompts

#Vector Operations and Embeddings
from langchain_openai import OpenAIEmbeddings    #Convert text to numerical vectors
from langchain_community.vectorstores import FAISS    #Vector database for similarity search
from langchain.schema import Document    #Document structure for text processing

#Retrieval-Augmented Generation (RAG)
from langchain.chains import RetrievalQA    #Question-answering chain with document retrieval

# 2: Securely Set API Keys
This section implements security best practices for handling sensitive API credentials.

**🔑 Required API Keys**

**OpenAI API Key:**\
Purpose: Powers GPT-3.5-turbo for sentiment analysis, summarization, and question-answering\
Format: Starts with sk- followed by alphanumeric characters\
Cost: Pay-per-use based on tokens processed

**Google Maps API Key:**\
Purpose: Fetches real restaurant reviews from Google Maps Places API\
Services needed: Places API with reviews access\
Cost: Free tier available, then pay-per-request

In [5]:
#Securely get API Keys using getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

if "Maps_API_KEY" not in os.environ:
    os.environ["Maps_API_KEY"] = getpass("Enter your Google Maps API key: ")

Enter your OpenAI API key: ··········
Enter your Google Maps API key: ··········


# 3: Define the Data Structure
This section establishes a structured data model for restaurant reviews using  @dataclass decorator. Creating a well-defined data structure is fundamental to building AI applications that can process information consistently and reliably.

In [6]:
@dataclass
class ReviewData:
    """Simple review data structure."""
    restaurant_name: str
    review_text: str
    rating: float
    reviewer_name: str

# 4: Function to Fetch Reviews from Google Maps
This cell contains the function that does the heavy lifting of talking to the Google Maps API. It works in three steps:

**Text Search:** It takes your search query (e.g., "best pizza in Shah Alam") and asks Google for a list of matching restaurants.

**Place Details:** For each restaurant found, it makes a second API call to get specific details, most importantly, the user reviews.

**Formatting:** It organizes the fetched data into the ReviewData structure we defined in the previous cell.

In [7]:
def get_Maps_reviews(api_key: str, query: str) -> List[ReviewData]:
    """Fetches restaurant reviews using the Google Maps Places API."""
    print(f"Searching for restaurants matching: '{query}'...")
    places_data = []

    # Step 1: Find a list of places (restaurants)
    search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    search_params = {"query": query, "type": "restaurant", "key": api_key}

    try:
        response = requests.get(search_url, params=search_params)
        response.raise_for_status()  # Raise an exception for bad status codes
        search_results = response.json().get("results", [])

        if not search_results:
            print("No restaurants found for your query.")
            return []

        # Step 2: For each place, get its details (including reviews)
        details_url = "https://maps.googleapis.com/maps/api/place/details/json"
        for place in search_results[:5]:  # Get details for the first 5 results
            place_id = place.get("place_id")
            if not place_id:
                continue

            details_params = {
                "place_id": place_id,
                "fields": "name,review,rating",
                "key": api_key,
            }
            details_response = requests.get(details_url, params=details_params)
            details_response.raise_for_status()
            place_details = details_response.json().get("result", {})

            # Step 3: Format the reviews into ReviewData objects
            restaurant_name = place_details.get("name")
            reviews = place_details.get("reviews", [])
            for review in reviews:
                if review.get("text"):  # Only use reviews with text
                    places_data.append(
                        ReviewData(
                            restaurant_name=restaurant_name,
                            review_text=review["text"],
                            rating=float(review.get("rating", 0)),
                            reviewer_name=review.get("author_name", "Anonymous"),
                        )
                    )

        print(f"Successfully fetched {len(places_data)} reviews from up to 5 restaurants.")
        return places_data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred with the API request: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# 5: The AI Reviewer Class
This is the core of our application, where all the AI logic lives. The SimplifiedRestaurantReviewer class is initialized with your OpenAI API key and sets up the three different AI capabilities:

**init:** The constructor that initializes the LangChain language model (ChatOpenAI) and the text embedding model (OpenAIEmbeddings).

**_setup_prompts:** This method creates the custom instructions (prompts) that we will send to the AI for sentiment analysis and summarization.

**analyze_sentiment, summarize_reviews, and ask_question:** These are the methods that execute each of the three AI tasks.

In [8]:
class SimplifiedRestaurantReviewer:
    """Simplified Restaurant Reviewer with 3 GenAI Capabilities."""

    def __init__(self, openai_api_key: str):
        self.openai_api_key = openai_api_key
        self.llm = ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo", api_key=self.openai_api_key)
        self.embeddings = OpenAIEmbeddings(api_key=self.openai_api_key)
        self.vector_store = None
        self.qa_chain = None
        self._setup_prompts()

    def _setup_prompts(self):
        sentiment_examples = [
            {"review": "The pasta was divine!", "sentiment": "POSITIVE - Score: 0.9"},
            {"review": "Food was terrible, service slow.", "sentiment": "NEGATIVE - Score: -0.8"},
            {"review": "It was okay. Nothing special.", "sentiment": "NEUTRAL - Score: 0.1"}
        ]
        sentiment_template = PromptTemplate(input_variables=["review", "sentiment"], template="Review: {review}\nSentiment: {sentiment}")
        self.sentiment_prompt = FewShotPromptTemplate(
            examples=sentiment_examples,
            example_prompt=sentiment_template,
            prefix="Analyze the sentiment of restaurant reviews:\n",
            suffix="Review: {review}\nSentiment:",
            input_variables=["review"]
        )
        self.summary_prompt = PromptTemplate(
            input_variables=["reviews"],
            template="""Analyze these restaurant reviews and provide a structured summary in JSON format:
Reviews:
{reviews}
Please respond with ONLY a JSON object in this exact format:
{{"PROS": ["list of positive aspects"],"CONS": ["list of negative aspects"],"Overall Summary": "brief overall summary\n","Recommendation Score": number_from_1_to_10}}"""
        )

    def analyze_sentiment(self, review_text: str) -> Dict:
        """Use few-shot prompting to analyze sentiment."""
        prompt = self.sentiment_prompt.format(review=review_text)
        response = self.llm.invoke(prompt)

        # Extract sentiment from the response
        response_text = response.content.strip()

        # Parse the sentiment (POSITIVE, NEGATIVE, or NEUTRAL)
        sentiment = "Unknown"
        if "POSITIVE" in response_text.upper():
            sentiment = "POSITIVE"
        elif "NEGATIVE" in response_text.upper():
            sentiment = "NEGATIVE"
        elif "NEUTRAL" in response_text.upper():
            sentiment = "NEUTRAL"

        return {
            "sentiment": sentiment,
            "explanation": response_text
        }

    def summarize_reviews(self, reviews: List[ReviewData]) -> Dict:
        """Use structured output to summarize multiple reviews."""
        review_texts = "\n".join([f"Review: {r.review_text} (Rating: {r.rating}/5)" for r in reviews])
        prompt = self.summary_prompt.format(reviews=review_texts)
        response = self.llm.invoke(prompt)
        try:
            return json.loads(response.content)
        except json.JSONDecodeError:
            return {"error": "Failed to parse JSON response", "raw_response": response.content}

    def setup_rag(self, reviews: List[ReviewData]):
        """Setup RAG system with review documents."""
        documents = [Document(page_content=rev.review_text, metadata={"restaurant": rev.restaurant_name, "rating": rev.rating, "reviewer": rev.reviewer_name}) for rev in reviews]
        self.vector_store = FAISS.from_documents(documents, self.embeddings)
        self.qa_chain = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=self.vector_store.as_retriever())
        print(f"RAG system setup complete with {len(reviews)} reviews.")

    def ask_question(self, question: str) -> str:
        """Use RAG to answer questions about restaurants."""
        if not self.qa_chain:
            return "RAG system not initialized."
        response = self.qa_chain.invoke({"query": question})
        return response['result']

# 6: Initialize and Fetch Data

Now we start the main process. This cell creates an instance of our SimplifiedRestaurantReviewer class and then calls the get_Maps_reviews function to fetch live data. You can input the search_query to find any type of food or restaurant you're interested in.

In [13]:
# Step 1: Initialization and Data Fetching
print("-" * 50)

# Initialize the core AI class
reviewer = SimplifiedRestaurantReviewer(os.environ["OPENAI_API_KEY"])

# Prompt user for search query
search_query = input("Enter your restaurant search query (e.g., 'best pizza in Shah Alam'): ")

# Fetch reviews from Google Maps Places API
all_reviews = get_Maps_reviews(os.environ["Maps_API_KEY"], search_query)
print("-" * 50)

--------------------------------------------------
Enter your restaurant search query (e.g., 'best pizza in Shah Alam'): best nasi dagang in shah alam
Searching for restaurants matching: 'best nasi dagang in shah alam'...
Successfully fetched 25 reviews from up to 5 restaurants.
--------------------------------------------------


# 7: Run AI Capability 1 - Sentiment Analysis

This cell demonstrates the first AI capability. It takes the very first review from our fetched data and sends it to the analyze_sentiment method. The AI then uses the few-shot examples we gave it to classify the sentiment and provide an explanation.

In [14]:
#AI Capability 1: Few-Shot Sentiment Analysis
if all_reviews:
    print("\nCAPABILITY 1: Few-Shot Sentiment Analysis")
    print("="*40)

    #Analyze sentiment for multiple reviews (up to 3)
    num_reviews_to_analyze = min(3, len(all_reviews))

    for i in range(num_reviews_to_analyze):
        review = all_reviews[i]
        print(f"\n---Review {i+1}---")
        print(f"Restaurant: '{review.restaurant_name}'")
        print(f"Review Text: '{review.review_text[:200]}{'...' if len(review.review_text) > 200 else ''}'")

        sentiment_result = reviewer.analyze_sentiment(review.review_text)
        print(f"Sentiment: {sentiment_result.get('sentiment', 'Unknown')}")
        print(f"AI Explanation: {sentiment_result['explanation']}")

        if i < num_reviews_to_analyze - 1:  # Don't print separator after last review
            print("-" * 30)

    print("-" * 50)
else:
    print("Skipping analysis because no reviews were fetched.")


CAPABILITY 1: Few-Shot Sentiment Analysis

---Review 1---
Restaurant: 'Baam Nasi Dagang Terengganu'
Review Text: 'Authentic taste of Terengganu!
Nasi dagang and nasi minyak was very aromatic and appetizing. Love the ambience. Small yet comfortable and clean warong. Do come early especially on weekends.'
Sentiment: POSITIVE
AI Explanation: POSITIVE - Score: 0.8
------------------------------

---Review 2---
Restaurant: 'Baam Nasi Dagang Terengganu'
Review Text: 'I always prefer nasi dagang terengganu compared to kelantan one mainly because of the existence of the acar timun lol and also the kuah lah i guess.. In KL, my favorite is quiet farrrr and the deliver...'
Sentiment: POSITIVE
AI Explanation: POSITIVE - Score: 0.7
------------------------------

---Review 3---
Restaurant: 'Baam Nasi Dagang Terengganu'
Review Text: 'Overall very tasty. I grew up in Terengganu eating nasi minyak for the past 32 years and I can tell the authenticity of this one. The shop setup is very comfortable an

# 8: Run AI Capability 2 - Structured Summary

Here, we demonstrate the second capability. The code calls the summarize_reviews method, passing in the entire list of fetched reviews. It then prints the AI-generated summary, which is formatted as a clean JSON object, making it easy to read and use.

In [15]:
#AI Capability 2: Structured Summary
if all_reviews:
    print("\nCAPABILITY 2: Structured Summary")
    print("="*40)
    print("Generating a summary of all fetched reviews...")
    summary_result = reviewer.summarize_reviews(all_reviews)
    #Print the JSON summary
    print(json.dumps(summary_result, indent=2))
    print("-" * 50)


CAPABILITY 2: Structured Summary
Generating a summary of all fetched reviews...
{
  "PROS": [
    "Authentic taste of Terengganu dishes",
    "Comfortable and clean ambiance",
    "Generous portions of food",
    "Variety of local Malay dishes",
    "Reasonable prices",
    "Friendly owner",
    "Good selection of meals",
    "Delicious food with big portions",
    "Easy access for online ordering"
  ],
  "CONS": [
    "Confusing pricing for some dishes",
    "Expensive pricing for certain items",
    "Long queues during peak hours",
    "Inconsistent service quality"
  ],
  "Overall Summary": "The restaurant offers authentic and delicious Terengganu and Kelantanese dishes with generous portions at reasonable prices. The ambiance is comfortable and clean, but there may be long queues during peak hours. Some customers have experienced confusing pricing and inconsistent service quality.",
  "Recommendation Score": 8
}
--------------------------------------------------


# 9: Run AI Capability 3 - RAG Q&A

Finally, this cell showcases Retrieval-Augmented Generation (RAG).

**setup_rag:** First, it builds a searchable "knowledge base" from all the reviews we fetched.

**ask_question:** Then, it asks a specific question. The AI will search through the reviews to find the relevant information and use it to generate a factual answer.

In [20]:
#AI Capability 3: Retrieval-Augmented Generation (RAG)
if all_reviews:
    print("\nCAPABILITY 3: Retrieval-Augmented Generation (RAG)")
    print("="*40)
    #Setup the RAG system with the fetched reviews
    reviewer.setup_rag(all_reviews)

    #Prompt user for question
    print("Examples of questions you can ask:")
    print("- Which place has the best sambal or is mentioned most positively?")
    print("- What are the common complaints about service?")
    print("- Which restaurant has the most authentic taste?")
    print("- What do customers say about pricing?")
    print()

    question = input("Enter your question about the restaurant reviews: ")
    print(f"\nAsking question: '{question}'")
    rag_answer = reviewer.ask_question(question)
    print("AI Answer:", rag_answer)
    print("-" * 50)

    #Footer
    #Notebook prepared by: Puteri Irene Alia binti Abdul Hadi (puteriirene@gmail.com)


CAPABILITY 3: Retrieval-Augmented Generation (RAG)
RAG system setup complete with 25 reviews.
Examples of questions you can ask:
- Which place has the best sambal or is mentioned most positively?
- What are the common complaints about service?
- Which restaurant has the most authentic taste?
- What do customers say about pricing?

Enter your question about the restaurant reviews: which -place suitable for kids?

Asking question: 'which -place suitable for kids?'
AI Answer: Based on the context provided, it seems like the restaurant mentioned is suitable for kids. They offer a variety of local Malay dishes, including snacks and soups, which could be appealing to children. However, it's always a good idea to check if they have specific kid-friendly options on their menu or if they provide any facilities for children.
--------------------------------------------------
